<a href="https://colab.research.google.com/github/adepril/SentimentsAnalysis/blob/main/FrenchSentimentAnalysisUsingTextBlob_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/fedi1996/french-sentiment-analysis-using-textblob

In [1]:
!pip install textblob_fr

In [2]:
import pandas as pd 
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
import re
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS
import string
from textblob import Blobber
from textblob_fr import PatternTagger, PatternAnalyzer
tb = Blobber(pos_tagger=PatternTagger(), analyzer=PatternAnalyzer())
import plotly.graph_objects as go
import plotly.express as px

In [3]:
#chargement de commentaires des clients de compagnies d'assurance
data = pd.read_csv('https://raw.githubusercontent.com/adepril/datasets/main/insurance-reviews-france-Comments.csv')
data.head()


,Unnamed: 0,Name,Comment,Month,Year
0,0,Réassurez-moi,Ravi ! La recherche et la comparaison des mutu...,8,2019
1,1,Réassurez-moi,Très satisfaite ! Un conseiller à l'écoute et ...,8,2019
2,2,Réassurez-moi,Je vous avais contacté pour la mise en place e...,8,2019
3,3,Réassurez-moi,Quelques mots sur Réassurez-moi. Je suis très ...,8,2019
4,4,Réassurez-moi,Votre cabinet m'a permis de pouvoir bénéficier...,8,2019


In [4]:
data= data.drop(['Unnamed: 0'],axis=1)

In [5]:
#calcule le pourcentage de "null" par colonne
NAN = [(c, data[c].isna().mean()*100) for c in data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN.sort_values("percentage", ascending=False)
NAN

,column_name,percentage
0,Name,0.00000
1,Comment,0.14523
2,Month,0.00000
3,Year,0.00000


In [6]:
data.shape

(11017, 4)

In [7]:
data =data.dropna()

In [8]:
NAN = [(c, data[c].isna().mean()*100) for c in data]
NAN = pd.DataFrame(NAN, columns=["column_name", "percentage"])
NAN.sort_values("percentage", ascending=False)

,column_name,percentage
0,Name,0.0
1,Comment,0.0
2,Month,0.0
3,Year,0.0


In [9]:
data.shape

(11001, 4)

Résultat : Il y a eu 16 lignes de suprimées

In [10]:
#convert data in lowercase
data["Comment"]= data["Comment"].str.lower()

# Word Tokenization and deleting punctuation

In [11]:
AComment=[]
for comment in data["Comment"].apply(str):
    Word_Tok = []
    for word in  re.sub("\W"," ",comment ).split():
        Word_Tok.append(word)
    AComment.append(Word_Tok)

#Ajoute une nouvelle colonne
data["Word_Tok"]= AComment
data.head()

,Name,Comment,Month,Year,Word_Tok
0,Réassurez-moi,ravi ! la recherche et la comparaison des mutu...,8,2019,"[ravi, la, recherche, et, la, comparaison, des..."
1,Réassurez-moi,très satisfaite ! un conseiller à l'écoute et ...,8,2019,"[très, satisfaite, un, conseiller, à, l, écout..."
2,Réassurez-moi,je vous avais contacté pour la mise en place e...,8,2019,"[je, vous, avais, contacté, pour, la, mise, en..."
3,Réassurez-moi,quelques mots sur réassurez-moi. je suis très ...,8,2019,"[quelques, mots, sur, réassurez, moi, je, suis..."
4,Réassurez-moi,votre cabinet m'a permis de pouvoir bénéficier...,8,2019,"[votre, cabinet, m, a, permis, de, pouvoir, bé..."


# StopWords

In [12]:
# set of Spacy's default stop words and delete negation words
stop_words=set(STOP_WORDS)

deselect_stop_words = ['n\'','ne','pas','plus','personne','aucun','ni','aucune','rien']
for w in deselect_stop_words:
    if w in stop_words:
        stop_words.remove(w)
    else:
        continue

In [13]:
# Delete stopwords
AllfilteredComment=[]
for comment in data["Word_Tok"]:
    filteredComment = [w for w in comment if not ((w in stop_words) or (len(w) == 1))]
    AllfilteredComment.append(' '.join(filteredComment))

In [14]:
# Ajoute nouvelle colonne : Commentaires après le preprocessing
data["CommentAferPreproc"]=AllfilteredComment
data.head()

,Name,Comment,Month,Year,Word_Tok,CommentAferPreproc
0,Réassurez-moi,ravi ! la recherche et la comparaison des mutu...,8,2019,"[ravi, la, recherche, et, la, comparaison, des...",ravi recherche comparaison mutuelles simples r...
1,Réassurez-moi,très satisfaite ! un conseiller à l'écoute et ...,8,2019,"[très, satisfaite, un, conseiller, à, l, écout...",satisfaite conseiller écoute sympathique bon p...
2,Réassurez-moi,je vous avais contacté pour la mise en place e...,8,2019,"[je, vous, avais, contacté, pour, la, mise, en...",contacté mise place janvier contrat mutuelle s...
3,Réassurez-moi,quelques mots sur réassurez-moi. je suis très ...,8,2019,"[quelques, mots, sur, réassurez, moi, je, suis...",mots réassurez satisfait services cabinet trou...
4,Réassurez-moi,votre cabinet m'a permis de pouvoir bénéficier...,8,2019,"[votre, cabinet, m, a, permis, de, pouvoir, bé...",cabinet permis pouvoir bénéficier contrat comp...


# Sentiment Analysis with TextBlob

In [15]:
senti_list = []
for i in data["CommentAferPreproc"]:
    vs = tb(i).sentiment[0]
    if (vs > 0):
        senti_list.append('Positive')
    elif (vs < 0):
        senti_list.append('Negative')
    else:
        senti_list.append('Neutral')   

 # Ajoute une colonne : Sentiment
data["sentiment"]=senti_list
data.head()       

,Name,Comment,Month,Year,Word_Tok,CommentAferPreproc,sentiment
0,Réassurez-moi,ravi ! la recherche et la comparaison des mutu...,8,2019,"[ravi, la, recherche, et, la, comparaison, des...",ravi recherche comparaison mutuelles simples r...,Positive
1,Réassurez-moi,très satisfaite ! un conseiller à l'écoute et ...,8,2019,"[très, satisfaite, un, conseiller, à, l, écout...",satisfaite conseiller écoute sympathique bon p...,Positive
2,Réassurez-moi,je vous avais contacté pour la mise en place e...,8,2019,"[je, vous, avais, contacté, pour, la, mise, en...",contacté mise place janvier contrat mutuelle s...,Positive
3,Réassurez-moi,quelques mots sur réassurez-moi. je suis très ...,8,2019,"[quelques, mots, sur, réassurez, moi, je, suis...",mots réassurez satisfait services cabinet trou...,Positive
4,Réassurez-moi,votre cabinet m'a permis de pouvoir bénéficier...,8,2019,"[votre, cabinet, m, a, permis, de, pouvoir, bé...",cabinet permis pouvoir bénéficier contrat comp...,Positive




---
# Data Visualization

---




In [16]:
Number_sentiment= data.groupby(["sentiment"])["Name"].count().reset_index().reset_index(drop=True)

In [17]:
fig = px.histogram(data, x="sentiment",color="sentiment")
fig.update_layout(
    title_text='Sentiment of reviews', # title of plot
    xaxis_title_text='Sentiment', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()

In [18]:
fig = px.pie(Number_sentiment, values=Number_sentiment['Name'], names=Number_sentiment['sentiment'], color_discrete_sequence=px.colors.sequential.Emrld
)
fig.show()

In [19]:
fig = px.histogram(data, x="Name",color="Name")
fig.update_layout(
    title_text='Nombre de commentaires par assurance', 
    xaxis_title_text='Assurance', 
    yaxis_title_text='Nombre de commentaires', 
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()

In [20]:
fig = px.histogram(data, x="Year",color="Year")
fig.update_layout(
    title_text='Nombre de commentaires par année', 
    xaxis_title_text='Année', 
    yaxis_title_text='Nombre de commentaires', 
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()

In [21]:
fig = px.histogram(data, x="Month",color="Month")
fig.update_layout(
    title_text='Nombre de commentaires par Mois', 
    xaxis_title_text='Mois', 
    yaxis_title_text='Nombre de commentaires', 
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()

In [22]:
fig = px.histogram(data, x="Name",color="sentiment")
fig.update_layout(
    title_text='Sentiments par Assurance', 
    xaxis_title_text='Assurance', 
    yaxis_title_text='Nombre de Commentaires', 
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()

In [23]:
fig = px.histogram(data, x="Year",color="sentiment")
fig.update_layout(
    title_text='Sentiments par Année', 
    xaxis_title_text='Année', 
    yaxis_title_text='Nombre de Commentaires', 
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()

In [24]:
fig = px.histogram(data, x="Month",color="sentiment")
fig.update_layout(
    title_text='Sentiments par Mois',
    xaxis_title_text='Mois', 
    yaxis_title_text='Nombre de Commentaires', 
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()

# Sentiment analysis in 2020

In [25]:
Data_2020 = data [(data['Year'] == 2020) ].reset_index(drop=True)
Data_2020['Month'].value_counts()

2    258
1    196
3     39
Name: Month, dtype: int64

Remarque : il n'y a que 3 mois

In [26]:
fig = px.histogram(Data_2020, x="sentiment",color="sentiment")
fig.update_layout(
    title_text='Sentiment of reviews in 2020', 
    xaxis_title_text='Sentiment', 
    yaxis_title_text='Count', 
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()

In [27]:
Number_sentiment_2020= Data_2020.groupby(["sentiment"])["Name"].count().reset_index().reset_index(drop=True)

fig = px.pie(Number_sentiment_2020, values=Number_sentiment_2020['Name'], names=Number_sentiment_2020['sentiment'], color_discrete_sequence=px.colors.sequential.Emrld)
fig.show()


In [28]:
fig = px.histogram(Data_2020, x="Name",color="sentiment")
fig.update_layout(
    title_text='Sentiments par Assurance en 2020', 
    xaxis_title_text='Assurance', 
    yaxis_title_text='Nombre de Commentaires', 
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()

In [29]:
fig = px.histogram(Data_2020, x="Month",color="sentiment")
fig.update_layout(
    title_text='Sentiments par Mois en 2020', 
    xaxis_title_text='Mois', 
    yaxis_title_text='Nombre de Commentaires', 
    bargap=0.2, 
    bargroupgap=0.1
)
fig.show()